In [3]:
import math

def emo_lev(arousal,valence):    #Function that returns emotion and intensity
    r = 5
    x, y = valence-5, arousal-5
    dist = (x**2+y**2)**0.5

    #Calc theta in radian
    theta = math.atan(y/x)

    #Convert theta from rad to deg
    theta = 180 * theta/math.pi

    def lev(dist):
        if(dist<2.5):
            return "Low"
        #if(dist>1.67 and dist<3.34):
            #return "Medium"
        else:
            return "High"

    #Check which intensity
    levl = lev(dist)


    def conv_emo(theta):
        if(x>0 and y>0):  #1st Quadrant
            if theta <= 90:
                return "Joy or Happiness",1,theta  #1st Quad, 1st half
        else:
            if(x>0 and y<0): #4
                theta+=360
                quad = 4
                if theta <= 360:
                    return "Tenderness",4,theta

            else:
                if(x<0 and y>0): #2
                    theta+=180
                    quad = 2
                    if theta <= 180:
                        return "Anger or Fear",2,theta
                else:
                    if(x<0 and y<0): #3
                       theta+=180
                       if theta <= 270:
                            return "Sadness",3,theta


    #print("Theta before conversion: {}".format(theta))
    emo,quad,theta = conv_emo(theta)
    #print("Distance = {} \nTheta after conversion = {} \nquadrant = {} ".format(dist,theta,quad))
    return emo,levl,theta,quad    #END of emo_lev

#Input the valence, arousal values here.
valence_x, arousal_y = -3,-2
emotion, level, theta,quadrant = emo_lev(arousal_y,valence_x)

#Optional printing. Just return the values you need if you want.
print("Emotion    : "+emotion)
print("Intensity  : "+level)
print("Theta      : {} degree".format("%.2f"%theta))
print("Quadrant   : {}".format(quadrant))

Emotion    : Sadness
Intensity  : High
Theta      : 221.19 degree
Quadrant   : 3


In [4]:
from models.tcn import EEG_TCN
from dataset.DEAP_DATASET import ModularDeapDataset
from torch.utils.data import DataLoader
import torch
from util.train import select_last_elm

DATA_SET_PATH = 'dataset/'
deap_val = ModularDeapDataset(DATA_SET_PATH, train=False)
loader = DataLoader(deap_val, shuffle=True, batch_size=1)

INPUT_SIZE = 40
HIDDEN_SIZE1 = 64
HIDDEN_SIZE2 = 32
OUTPUT_SIZE = 4

MDL_TCN_PATH = 'models/saved_weights/tcn_deeper_v2.pth'

CHAN_LIST = [32, 24, 16, 10, 6, 2]
KERN_SIZE = 5
DROP_OUT = 0.2

model = EEG_TCN(CHAN_LIST,KERN_SIZE,DROP_OUT)

model.load_state_dict(torch.load(MDL_TCN_PATH, map_location="cpu"))
model.eval()

with torch.no_grad():
    for i, (signal, label) in enumerate(loader):
        output = model(signal)
        output = select_last_elm(output, "cpu")
        output = output.numpy()
        label = label.numpy()
        print("----------------------------------")

        emotion, level, theta,quadrant = emo_lev(label[0][1],label[0][0])
        print("ACTUAL",label)
        print("\tEmotion    : "+emotion)
        print("\tIntensity  : "+level)
        print("\tTheta      : {} degree".format("%.2f"%theta))
        print("\tQuadrant   : {}".format(quadrant))
        
        emotion, level, theta,quadrant = emo_lev(output[0][1],output[0][0])
        print("PREDICTED",output)
        print("\tEmotion    : "+emotion)
        print("\tIntensity  : "+level)
        print("\tTheta      : {} degree".format("%.2f"%theta))
        print("\tQuadrant   : {}".format(quadrant))

----------------------------------
ACTUAL [[7.03 4.99]]
	Emotion    : Tenderness
	Intensity  : Low
	Theta      : 359.72 degree
	Quadrant   : 4
PREDICTED [[3.590856  3.1105244]]
	Emotion    : Sadness
	Intensity  : Low
	Theta      : 233.28 degree
	Quadrant   : 3
----------------------------------
ACTUAL [[6.42 3.62]]
	Emotion    : Tenderness
	Intensity  : Low
	Theta      : 315.82 degree
	Quadrant   : 4
PREDICTED [[3.43317   3.0327005]]
	Emotion    : Sadness
	Intensity  : High
	Theta      : 231.46 degree
	Quadrant   : 3
----------------------------------
ACTUAL [[2.95 4.9 ]]
	Emotion    : Sadness
	Intensity  : Low
	Theta      : 182.79 degree
	Quadrant   : 3
PREDICTED [[2.345553  4.6164093]]
	Emotion    : Sadness
	Intensity  : High
	Theta      : 188.22 degree
	Quadrant   : 3
----------------------------------
ACTUAL [[4.96 5.01]]
	Emotion    : Anger or Fear
	Intensity  : Low
	Theta      : 165.96 degree
	Quadrant   : 2
PREDICTED [[5.6865587 5.593921 ]]
	Emotion    : Joy or Happiness
	Intens

<ipython-input-3-fd2d3ce34191>:9: RuntimeWarning: divide by zero encountered in double_scalars
  theta = math.atan(y/x)


TypeError: cannot unpack non-iterable NoneType object